# NYC Buildings
Written by Philipp Rudiger<br>
Created: January 27, 2021<br>
Last updated: April 20, 2022

Many plotting libraries can handle collections of polygons, including [Bokeh](https://docs.bokeh.org/en/latest/docs/gallery/texas.html) and [HoloViews](http://holoviews.org/gallery/demos/bokeh/choropleth_data_link.html). However, because browser-based libraries like Bokeh and Plotly send all the polygon data to JavaScript running in the browser, they can struggle when either the collections or the individual polygons themselves get large. Even natively in Python, typical formats like Shapely for representing polygons scale poorly to large polygon collections, because each polygon is wrapped up as a full, separate Python object, leading to a lot of duplicated storage overhead when many polygons of the same type are defined.

If you want to work with lots of polygons, here you can see how to use [SpatialPandas](https://github.com/holoviz/spatialpandas) and [Dask](https://dask.org) to represent large collections of polygons efficiently in memory, [fastparquet](https://fastparquet.readthedocs.io/) to represent them efficiently on disk, [Datashader](https://datashader.org) to render them quickly in a web browser, and [HoloViews](https://holoviews.org) to provide a convenient API. This notebook also demonstrates how to support hovering for datashaded polygons, with HoloViews setting up Bokeh to overlay a single vector-based representation of a polygon where the mouse cursor is, while all the rest are sent to the browser only as rendered pixels. That way hover and other interactive features can be supported fully without ever needing to transfer large amounts of data or store them in the limited memory of the web browser tab.  

This example plots the outlines of all one million+ buildings in New York City. See
[nyc.gov](https://www1.nyc.gov/site/doitt/residents/gis-2d-data.page) for the original data and its description.

In [ ]:
import holoviews as hv
import colorcet as cc
import datashader as ds
import spatialpandas as spd
import spatialpandas.io

from dask.diagnostics import ProgressBar
from holoviews.operation.datashader import rasterize, datashade, inspect_polygons

# Add more resolution to dynamic plots, particularly important for Retina displays
from holoviews.streams import PlotSize
PlotSize.scale=2.0

hv.extension('bokeh')

In [ ]:
ddf = spd.io.read_parquet_dask('./data/nyc_buildings.parq').persist()
print(len(ddf))
ddf.head()

Here you can see that we have 1.1 million "MultiPolygons", some of which have a `type` and `name` declared.

To get a look at this data, let's plot all the polygons, overlaid on a tiled map of the region:

In [ ]:
polys = hv.Polygons(ddf, vdims='type')
tiles = hv.element.tiles.CartoLight()
tiles * rasterize(polys, aggregator='any').opts(width=600, height=500)

At this scale, the plot looks like a bunch of dots or large colored areas, because each building is smaller than a pixel in the plot. But if you have a live Python server running, you can use the Bokeh tools to zoom in and have the plot dynamically redrawn, showing you the full outline of each polygon. You should see more detail whenever you zoom in, possibly after a short delay after Datashader re-renders the new scene.

Now let's make use of the category information. To get a manageable number of types, we'll compute the top 10 most common categories and drop everything else:

In [ ]:
cats = list(ddf.type.value_counts().compute().iloc[:10].index.values) + \
       ['unknown']

ddf['type'] = ddf.type.replace({None: 'unknown'})
ddf = ddf[ddf.type.isin(cats)]
ddf['type'] = ddf['type'].astype('category').cat.as_known()

SpatialPandas lets us build a spatial index for accessing spatially organized regions more quickly, so let's do that:

In [ ]:
with ProgressBar():
    ddf = ddf.build_sindex().persist()

Now we can view each category separately with a selector widget:

In [ ]:
polys = hv.Polygons(ddf, vdims='type')

hmap  = hv.HoloMap({ cat: polys.select(type=cat) for cat in cats}, 'Type')
rcats = rasterize(hmap, aggregator='any').opts(width=600, height=500)

tiles * rcats

If you look at each one, you can see that unfortunately most of the categories are unknown, but there are interesting patterns (e.g. almost no garages in Manhattan, and apparently all the sheds are in New Jersey).

Since these buildings don't normally overlap, we can actually combine them all into a single plot using color to show all of the categories (though we have to construct a color key manually):

In [ ]:
colors    = cc.glasbey_bw_minc_20_maxl_70
color_key = {cat: tuple(int(e*255.) for e in colors[i]) 
             for i, cat in enumerate(cats)}
legend    = hv.NdOverlay({k: hv.Points([(0,0)], label=str(k)).opts(
                                        color=cc.rgb_to_hex(*v), size=0, 
                                        apply_ranges=False)
                          for k, v in color_key.items()}, 'Type')

Now we let's show the color-coded plot, and use the `inspect_polygons` operation to allow us to hover on the data dynamically:

In [ ]:
shaded = datashade(polys, color_key=color_key, aggregator=ds.by('type', ds.any()))

hover = inspect_polygons(shaded).opts(fill_color='red', tools=['hover'])

plot = tiles * shaded * legend * hover
plot.opts(min_height=500, min_width=600, responsive=True)

If you zoom into an area of interest and then hover over the polygons with a mouse, you'll see an overlay of the building at that location, with hover information indicating its type and name when available. 

Finally, we'll make this notebook into a shareable app (run with `panel serve nyc_buildings.ipynb`, or `anaconda-project run dashboard`)

In [ ]:
text = """
# [1 million buildings in NYC](https://examples.pyviz.org/nyc_buildings)
## Rendered using [Datashader](https://datashader.org) and [HoloViews](https://holoviews.org).
"""

import panel as pn
pn.Column(text, pn.panel(plot, sizing_mode='stretch_width')).servable();